## TRABAJO PRACTICO 3: CHAT PARA CONSULTAR DISTINTOS CVs A TRAVES DE UN AGENTE
ALUMNA: Lara Rosenberg

In [1]:
#Importamos las librerias necesarias
import os
import streamlit as st
from pinecone import Pinecone
from groq import Groq
from typing import List, Set
import nltk
import re
import unicodedata
nltk.download("punkt_tab")
nltk.download("punkt")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\damia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\damia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Definimos las API KEY necesarias (Pinecone y Groq)
os.environ['PINECONE_API_KEY'] = ''
os.environ['GROQ_API_KEY'] = ''

In [3]:
# Funcion para particionar el texto
def read_and_chunk_sentences(
    file_path: str,
    chunk_size: int = 40,
    overlap: int = 10
) -> List[str]:
    """
    Lee el archivo de texto, lo separa en oraciones y hace el chunk con overlap.

    Argumentos:
        file_path (str): Path al archivo de texto.
        chunk_size (int): Numero de oraciones por chunk.
        overlap (int): Numero de oraciones que estan en overlap.

    Returns:
        List[str]: Lista de chunks.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} does not exist.")

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    sentences = nltk.sent_tokenize(text, language='spanish')
    chunks = []
    i = 0
    while i < len(sentences):
        chunk = sentences[i:i+chunk_size]
        if chunk:
            chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

In [4]:
# Instanciamos las API KEY necesarias
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [ ]:
# Inicializamos cliente Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Función para subir los documentos a Pinecone
def upload_documents(file_path: str, index_name: str, namespace: str):
    # Leemos el archivo y lo dividimos en chunks
    chunks = read_and_chunk_sentences(file_path, chunk_size=5, overlap=2)

    # Creamos los documentos en el formato requerido por Pinecone
    documents = []
    for i, chunk in enumerate(chunks):
        documents.append({
            "_id": f"{namespace}_chunk_{i + 1}",
            "chunk_text": chunk,
            "category": namespace  # Guardamos la categoría como el namespace
        })
    print(documents)

    # Conectamos al índice en Pinecone
    index_pc = pc.Index(index_name)

    # Subimos los documentos al índice
    index_pc.upsert_records(namespace=namespace, records=documents)
    print(f"Subidos {len(documents)} chunks al índice '{index_name}' en el namespace '{namespace}'.")

# Definimos nombres de los archivos y sus namespaces
files_and_namespaces = [
    ("CV LARA ROSENBERG.txt", "cv-lara-ns"),
    ("CV VICTORIA TERAN.txt", "cv-victoria-ns"),
    ("CV CLAUDIO BARRIL.txt", "cv-claudio-ns")
]

# Creamos un índice por cada CV y subimos los documentos a su respectivo índice
for file, namespace in files_and_namespaces:
    index_name = f"{namespace}-index"  # Usamos el namespace para crear un índice único para cada CV

    # Verificamos si el índice ya existe. Si no, lo creamos.
    if not pc.list_indexes() or index_name not in [i.name for i in pc.list_indexes()]:
        pc.create_index_for_model(
            name=index_name,
            cloud="aws",
            region="us-east-1",
            embed={
                "model": "llama-text-embed-v2",
                "field_map": {"text": "chunk_text"}
            }
        )
        print(f"Índice '{index_name}' creado.")

    # Subimos los documentos al índice específico
    #upload_documents(file, index_name, namespace)

In [ ]:
# Función para normalizar texto (quitar tildes, pasar a minúsculas)
def normalizar(texto: str) -> str:
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto)
    return ''.join(c for c in texto if unicodedata.category(c) != 'Mn')

# Patrones para identificar las personas mencionadas en la consulta
person_patterns = {
    "cv-lara-ns-index": re.compile(r"\blara(\s+rosenberg)?\b|\brosenberg\b", re.IGNORECASE),
    "cv-victoria-ns-index": re.compile(r"\bvictoria(\s+teran)?\b|\bteran\b", re.IGNORECASE),
    "cv-claudio-ns-index": re.compile(r"\bclaudio(\s+barril)?\b|\bbarril\b", re.IGNORECASE)
}

# Función para identificar las personas mencionadas --> devuelve un diccionario con los nombres de los indices de las personas que identifico
def identificar_personas_mencionadas(texto: str) -> Set[str]:
    texto_normalizado = normalizar(texto)
    indices_decididos = set()

    for idx, pat in person_patterns.items():
        if pat.search(texto_normalizado):
            indices_decididos.add(idx)

    return indices_decididos

# Función para buscar los chunks más relevantes en Pinecone
def search_similar(texto, top_k=10, indices=None):
    contexto = ""

    if not indices:
        indices = ["cv-lara-ns-index"]

    # Realizamos la búsqueda.
    for indice in indices:
        namespace = indice.replace("-index", "")
        results = pc.Index(indice).search(
            namespace=namespace,
            query={
                "top_k": top_k,
                "inputs": {
                    'text': texto
                }
            }
        )
        # Recopilamos los resultados y los agregamos al contexto
        contexto += "\n".join([hit['fields']['chunk_text'] for hit in results['result']['hits']])

    return contexto

# Definirmos el agente para generar la respuesta usando Groq
class Agent:
    def __init__(self, client, model="meta-llama/llama-4-scout-17b-16e-instruct"):
        self.client = client
        self.model = model
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def __call__(self, message):
        self.add_message("user", message)
        result = self.execute(message)
        self.add_message("assistant", result)
        return result

    def execute(self, message):
        # Detectamos la persona mencionada en la consulta
        indices_decididos = identificar_personas_mencionadas(message)

        # Si no se mencionan personas, utilizamos por default el cv de Lara
        if not indices_decididos:
            indices_decididos = ["cv-lara-ns-index"]

        # Buscamos el contexto relevante
        contexto = search_similar(message, indices=indices_decididos)

        # Generamos la respuesta con el contexto obtenido
        prompt = f"""
Sos un asistente conversacional. Respondé de forma natural y conversacional.

Responde basandote en el contexto. Si no tenes informacion, aclaralo.
Consulta previa (historial): {self.messages}

Consulta: {message}

Contexto:
{contexto}
"""
        self.add_message("user", prompt.strip())

        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=0.7
        )
        return completion.choices[0].message.content

# Inicializamos el agente con Groq
groq_client = Groq(api_key=GROQ_API_KEY)
agente = Agent(client=groq_client)


In [7]:
# Ejemplo de consulta
consulta_usuario = "En cuantas empresas trabajo Victoria?"
respuesta = agente(consulta_usuario)
print(respuesta)

C:\Users\damia\Anaconda3\envs\nuevo_entorno\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Según la información que tengo, Victoria trabaja o ha trabajado en al menos 3 empresas:

1. Mercado Libre (2022 - actualidad) como Analista Sr de Riesgo de Crédito
2. AFIP - Administración Federal de Ingresos Públicos (2019 - 2022) como Analista de Datos
3. HSBC (2015) como pasante

También tuvo un rol como Analista Administrativo Contable desde 2015 hasta 2019, pero no especifica si fue en una empresa en particular o si fue en alguna de las mencionadas anteriormente.


In [8]:
# Ejemplo de consulta
consulta_usuario = "En cuantas empresas trabajo Lara?"
respuesta = agente(consulta_usuario)
print(respuesta)

Según la información que tengo, Lara trabaja o ha trabajado en al menos 6 empresas:

1. Mercado Libre (Marzo 2023 - actualidad) en dos roles diferentes: 
   - Credit Risk Modeling Supervisor (Marzo 2024 - actualidad)
   - Credit Risk Modeling Sr Analyst (Marzo 2023 - Marzo 2024)

2. Algorithia (Noviembre 2021 - Marzo 2023) como Data Scientist

3. Círculo de Crédito (Marzo 2020 - Noviembre 2021) como Consultor de Modelos

4. Banco Patagonia (Septiembre 2019 - Marzo 2020) como Analista de Riesgos Financieros

5. Instituto Argentino de Mercado de Capitales (Agosto 2018 - Agosto 2019) como Analista Financiero

6. MetLife Seguros (Abril 2018 - Agosto 2018) como pasante

Espero que esta información sea útil. ¿Necesitas algo más?


In [9]:
# Ejemplo de consulta
consulta_usuario = "Hay alguna empresa en comun entre Lara, Victoria y Claudio?"
respuesta = agente(consulta_usuario)
print(respuesta)

Sí, hay una empresa en común entre Lara, Victoria y Claudio.

**Mercado Libre** es la empresa que tienen en común.

* Lara trabaja en Mercado Libre desde Marzo 2023 como Credit Risk Modeling Supervisor.
* Victoria trabaja en Mercado Libre desde 2022 como Analista Sr de Riesgo de Crédito.
* Claudio trabajó en Mercado Libre desde Marzo 2019 hasta Diciembre 2021 como Líder Técnico de Desarrollo de Software y luego como Ingeniero de Software Senior.
